In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

conv_lay_1 = 32
stride_lay_1 = 1
conv_lay_1_k = 5
max_pool_lay_1_k = 2

conv_lay_2 = 64
stride_lay_2 = 1
conv_lay_2_k = 5
max_pool_lay_2_k = 2

n_hidden = 1024
n_out = 10

input_hidden_layer = (input_width//(max_pool_lay_1_k*max_pool_lay_2_k))* (input_height//(max_pool_lay_1_k*max_pool_lay_2_k)) *conv_lay_2

In [ ]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([conv_lay_1_k, conv_lay_1_k, input_channels, conv_lay_1])),
    'wc2' : tf.Variable(tf.random_normal([conv_lay_2_k, conv_lay_2_k, conv_lay_1, conv_lay_2])),
    'wh'  : tf.Variable(tf.random_normal([input_hidden_layer, n_hidden])),
    'wo'  : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([conv_lay_1])),
    'bc2' : tf.Variable(tf.random_normal([conv_lay_2])),
    'bh'  : tf.Variable(tf.random_normal([n_hidden])),
    'bo'  : tf.Variable(tf.random_normal([n_out]))
}

In [ ]:
def conv(x, weight, bias, stri):
    out = tf.nn.conv2d(x, weight, padding='SAME', strides=[1, stri, stri, 1])
    out = tf.add(out, bias)
    out = tf.nn.relu(out)
    return out

def pool(x, k):
    return tf.nn.max_pool(x, padding='SAME', ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [ ]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, [-1, input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_lay_1)
    conv_pool1 = pool(conv1, max_pool_lay_1_k)
    
    conv2 = conv(conv_pool1, weights['wc2'], biases['bc2'], stride_lay_2)
    conv_pool2 = pool(conv2, max_pool_lay_2_k)
    
    x_reshaped = tf.reshape(conv_pool2, shape = [-1, input_hidden_layer])
    hidden_output_1 = tf.add(tf.matmul(x_reshaped, weights['wh']), biases['bh'])
    hidden_output_activated = tf.nn.relu(hidden_output_1)
    hidden_output_dropout = tf.nn.dropout(hidden_output_activated, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output_dropout, weights['wo']), biases['bo'])
    return output

In [ ]:
x = tf.placeholder('float', [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder('float')

In [ ]:
pred = cnn(x, weights, biases, keep_prob)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:1.0})
        total_cost += c
    print(total_cost)

In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob: 1.0})
correct_predictions.sum()